Calculating Mean Average Precision from yolo format txt files

In [1]:
import glob,os
import numpy as np
import supervision as sv
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
pd.set_option('mode.chained_assignment', None)

In [2]:
# ground truth path
gt_path = '/home/ruser/data_download_experiments/sentinel_data/crossval/lucknow_sarath_grid_aa_v3/'
# prediction path
pred_path = '/home/vuser/brick_kilns/yolov10/'

In [3]:
def box_iou_batch(
	boxes_a: np.ndarray, boxes_b: np.ndarray
) -> np.ndarray:

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])

    area_a = box_area(boxes_a.T)
    area_b = box_area(boxes_b.T)

    top_left = np.maximum(boxes_a[:, None, :2], boxes_b[:, :2])
    bottom_right = np.minimum(boxes_a[:, None, 2:], boxes_b[:, 2:])

    area_inter = np.prod(
    	np.clip(bottom_right - top_left, a_min=0, a_max=None), 2)
        
    return area_inter / (area_a[:, None] + area_b - area_inter)

In [4]:
def non_max_suppression(
   predictions: np.ndarray, iou_threshold: float = 0.5
) -> np.ndarray:
    rows, columns = predictions.shape
    sort_index = np.flip(predictions[:, 5].argsort())
    predictions = predictions[sort_index]
    boxes = predictions[:, :4]
    categories = predictions[:, 4]
    ious = box_iou_batch(boxes, boxes)
    ious = ious - np.eye(rows)

    keep = np.ones(rows, dtype=bool)

    for index, (iou, category) in enumerate(zip(ious, categories)):
        if not keep[index]:
            continue

        condition = (iou > iou_threshold) & (categories == category)
        keep = keep & ~condition

    return keep[sort_index.argsort()]


In [5]:
def xyxy(arr, type = None):
    # arr -- numpy array (n x 6)
    tmp = []
    for x in arr:
        actual_w = x[3] * 1120
        actual_h = x[4] * 1120
        actual_x_c = x[1] * 1120
        actual_y_c = x[2] * 1120
        xmin , ymin =  actual_x_c - (actual_w/2), actual_y_c - (actual_h/2)
        xmax, ymax = actual_x_c + (actual_w/2), actual_y_c + (actual_h/2)
        if type == 'P':
            tmp.append([xmin, ymin, xmax, ymax, x[0], x[-1]]) # xmin, xmax, ymin,ymax, class, conf
        else:
            tmp.append([xmin, ymin, xmax, ymax, x[0]])

    return np.array(tmp)
    

In [6]:
df = pd.DataFrame(columns = ['map50','map75','map50-95'], index = ['overall', 'FCBK', 'Zigzag'])

In [7]:
a = []
nms = False
num_folds = 4
for fold in range(num_folds):
    gt_dir = f'{gt_path}/{fold}/test/labels'
    pred_dir = f'{pred_path}/lucknow_sentinel_pred/{fold}-2'

    # gt_dir = f'{gt_path}/{fold}/val/labels/'
    # pred_dir = f'{pred_path}/'

    gt_files = glob.glob(os.path.join(gt_dir,'*.txt'))
    pred_files = glob.glob(os.path.join(pred_dir,'*.txt'))
    # gt_files,pred_files = gt_files[:1], pred_files[:1]

    gt_results = []
    pred_files = []
    pred_results = []
    info = False

    for i in range(len(gt_files)):
        # gt_results.append(xyxy(np.loadtxt(gt_files[i], ndmin= 2)))
        file_name = os.path.basename(gt_files[i])
        corresponding_pred_file = os.path.join(pred_dir, file_name)
        if os.path.exists(corresponding_pred_file):
            pred_files.append(corresponding_pred_file)
            gt_results.append(xyxy(np.loadtxt(gt_files[i], ndmin= 2)))
        # else:
        #     pred_files.append(None)

    # for f in pred_files:
    #     if f == None:
    #         print(f)

    if nms:
        for i in range(len(pred_files)):
            # if pred_files[i] == None:
            #     continue
            results = np.loadtxt(pred_files[i], ndmin= 2) # load all predictions of single image
            bool_list = non_max_suppression(xyxy(results, type = 'P')) #apply non max suppression
            best_boxes = []
            for i, element in enumerate(bool_list):
                if element:
                    best_boxes.append(results[i]) # add bbox with highest IOU and highest score
            pred_results.append(xyxy(best_boxes, type = 'P'))
            

    else:
        for i in range(len(pred_files)):
            if pred_files[i] == None:
                continue
            pred_results.append(xyxy(np.loadtxt(pred_files[i], ndmin= 2), type='P'))


    if info:
        print(f'No of GT images in Fold {fold}:', len(gt_results))
        print(f'No of bbox in one GT image in Fold {fold}', len(gt_results[0]), type(gt_results[0]))
        print('No of Pred images:', len(pred_results))
        print('No of predictions in one GT image', len(pred_results[0]), type(pred_results[0]))
        print('Shape of predictions in one GT image',pred_results[0].shape)

    #Finding mean average precision
    mean_average_precison = sv.MeanAveragePrecision.from_tensors(
                predictions= pred_results,
                targets= gt_results,
            )
    # print(mean_average_precison)
    map_fcbk , map_zigzag = mean_average_precison.per_class_ap50_95.mean(axis = 1)
    df = df.copy()
    df.loc['overall']['map50'] = mean_average_precison.map50
    df.loc['overall']['map75'] = mean_average_precison.per_class_ap50_95[:,5].mean()
    df.loc['overall']['map50-95'] = mean_average_precison.map50_95
    df.loc['FCBK']['map50'] = mean_average_precison.per_class_ap50_95[:,0][0]
    df.loc['FCBK']['map75'] = mean_average_precison.per_class_ap50_95[:,5][0]
    df.loc['FCBK']['map50-95'] = map_fcbk
    df.loc['Zigzag']['map50'] = mean_average_precison.per_class_ap50_95[:,0][1]
    df.loc['Zigzag']['map75'] = mean_average_precison.per_class_ap50_95[:,5][1]
    df.loc['Zigzag']['map50-95'] = map_zigzag
    print(f'Fold {fold}')
    print(df,end = '\n\n')

    a.append(df)
    # print(a)

Fold 0
            map50     map75  map50-95
overall  0.309407  0.042006   0.11775
FCBK     0.197696    0.0446  0.080082
Zigzag   0.421118  0.039412  0.155419

Fold 1
            map50     map75  map50-95
overall  0.355014   0.09625  0.136767
FCBK     0.190324  0.089904  0.093936
Zigzag   0.519703  0.102597  0.179597

Fold 2
            map50     map75  map50-95
overall  0.397598   0.06595  0.147619
FCBK     0.244181  0.027626  0.080733
Zigzag   0.551016  0.104274  0.214504

Fold 3
            map50     map75  map50-95
overall  0.401938  0.072842  0.144298
FCBK     0.247015  0.038776  0.098263
Zigzag    0.55686  0.106908  0.190332



In [8]:
print('Combined MAP of all folds ')
(a[0]+a[1]+a[2]+a[3])/num_folds


Combined MAP of all folds 


,map50,map75,map50-95
overall,0.365989,0.069262,0.136608
FCBK,0.219804,0.050226,0.088254
Zigzag,0.512174,0.088298,0.184963


In [10]:
# standard deviation
mean_and_std = {}
maps = ["map50", "map75", "map50-95"]
for map in maps:
    mean_and_std[map] = {}
    for i in range(2):
        if i==0:
            type = 'FCBK'
        else:
            type = 'Zigzag'
        values = []
        for j in range(4):
            values.append(a[j].loc[type][map])
        # print(values)
        mean_and_std[map][type] = (np.mean(values), np.std(values))


print(mean_and_std)



{'map50': {'FCBK': (0.2198040824145684, 0.025944617520327475), 'Zigzag': (0.5121741517090259, 0.054436837884349644)}, 'map75': {'FCBK': (0.05022648850825275, 0.02370587892358508), 'Zigzag': (0.08829754341697102, 0.028266035714385938)}, 'map50-95': {'FCBK': (0.088253703423568, 0.007996905067489433), 'Zigzag': (0.1849629572298537, 0.02123169878432321)}}


In [11]:
df2 = pd.DataFrame(mean_and_std, index = ['FCBK', 'Zigzag'])
df2

,map50,map75,map50-95
FCBK,"(0.2198040824145684, 0.025944617520327475)","(0.05022648850825275, 0.02370587892358508)","(0.088253703423568, 0.007996905067489433)"
Zigzag,"(0.5121741517090259, 0.054436837884349644)","(0.08829754341697102, 0.028266035714385938)","(0.1849629572298537, 0.02123169878432321)"
